In [1]:
import pandas as pd
import altair as alt
from vega_datasets import data
import numpy as np
alt.data_transformers.enable('default', max_rows=None) # remove max row cap

DataTransformerRegistry.enable('default')

In [2]:
ur = pd.read_csv('Unemployment_by_state.csv')
us_states = alt.topo_feature(data.us_10m.url, feature='states')

ur = ur.drop(0).reset_index() # remove US row

# filter for dates wanted
ur = ur[['Fips','2000','2001','2002','2003','2004','2005','2006',
         '2007','2008','2009','2010','2011','2012','2013','2014',
         '2015','2016','2017','2018','Area']]

ur = ur.dropna()

# get fips to match geo data
ur['Fips'] = (ur['Fips'] / 1000).astype(int)

# rename to match geo data
ur.rename(columns={'Fips': 'id'}, inplace=True)

# change formating 
ur = ur.melt(id_vars='id', var_name='year', value_name='unemployment_rate')

ur = ur.pivot(index='id', columns='year', values='unemployment_rate').reset_index()
columns = [str(year) for year in range(2000, 2019)]

min_rate = ur[columns].min().min()
max_rate = ur[columns].max().max()

slider = alt.binding_range(min=2000, max=2018, step=1)
select_year = alt.selection_single(name='select',fields=['year'],
                                   bind=slider, init={'year': 2000})



ur_by_state = alt.Chart(us_states).mark_geoshape().project(
    type='albersUsa'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(ur, 'id', ['unemployment_rate'] + columns + ['Area'])  # include 'Area' for tool tip
).transform_fold( 
    columns, as_=['year', 'unemployment_rate']
).transform_calculate(
    year='parseInt(datum.year)',
    unemployment_rate='isValid(datum.unemployment_rate) ? datum.unemployment_rate : -1'
).encode(
    color=alt.condition(
        'datum.unemployment_rate > 0',
        alt.Color('unemployment_rate:Q', 
                  scale=alt.Scale(scheme='blues',domain=[min_rate, max_rate]),
                  title='Unemployment Rate (%)'),
        alt.value('#dbe9f6')
    ),
    tooltip=[alt.Tooltip('Area:N', title='State'), 
             alt.Tooltip('unemployment_rate:Q', title='Unemployment Rate (%)')]
).add_selection(
    select_year
).properties(
    title='Unemployment Rate by State (2000-2018)',
    width=500,
    height=300
).transform_filter(
    select_year
).configure_view(
    stroke=None  # Removes the border
)

ur_by_state

/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)